In [1]:
pip install pyspark

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
if __name__ == "__main__":
 # create Spark context with Spark configuration
 conf = SparkConf().setAppName("Word Count -
Python").set("spark.hadoop.yarn.resourcemanager.address", 
"mbpdemaippe1507.home:4040")
 sc = SparkContext(conf=conf)
 # read in text file and split each document into words
 words = sc.textFile("/Users/marcphilippehuget/Downloads/spark.txt").flatMap(lambda line: line.split(" "))
 # count the occurrence of each word
 wordCounts = words.map(lambda word: (word, 1)).reduceByKey(lambda a,b:a +b)
 wordCounts.saveAsTextFile("/Users/marc-philippehuget/Downloads/pyoutput2"